**Author** : Kinza Kasmi  
**Date** : February 2021

This tutorial explains the basic features of Google Earth Engine and the Java as well as the Python associated syntax.

# Importing earth engine

In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

# Table of Contents
* [I. Quick Start](#I.-Quick-Start)
    * [1. Declaring Variables](#1.-Declaring-Variables)
    * [2. Filtering](#2.-Filtering)
    * [3. Mapping](#3.-Mapping)
    * [4. Displaying Metadata](#4.-Displaying-Metadata)
    * [5. Exporting Data](#5.-Exporting-Data)
    * [6. Example : Exporting a Sentinel1 tif Image](#6.-Example-:-Exporting-a-Sentinel1-tif-Image)
* [II. Satellite Images Export](#II.Satellite-Images-Export)
    * [1. Image Extraction](#1.-Image-Extraction)
    * [2. Region Definition](#2.-Region-Definition)
    * [3. Export to tif and kml](#3.-Export-to-tif-and-kml)
    * [4. Putting Everything Together](#4.-Putting-Everything-Together)
* [III. Earth Engine Code Editor Tutorials](#III.-Earth-Engine-Code-Editor-Tutorials)
    * [1. Add Table to Assets](#1.-Add-Table-to-Assets)
    * [2. Add Image to Assets](#2.-Add-Image-to-Assets)
    * [3. Adding Missclassified Pixels to Assets and Array
    ](#3.-Adding-Missclassified-Pixels-to-Assets-and-Array)

# I. Quick Start

# Table of Contents
* [1. Declaring Variables](#1.-Declaring-Variables)
* [2. Filtering](#2.-Filtering)
* [3. Mapping](#3.-Mapping)
* [4. Displaying Metadata](#4.-Displaying-Metadata)
* [5. Exporting Data](#5.-Exporting-Data)
* [6. Example : Exporting a Sentinel1 tif Image](#6.-Example-:-Exporting-a-Sentinel1-tif-Image)

## 1. Declaring Variables
```
var variableName = ee.ContainerType(value); #Java   
variableName = ee.ContainerType(value) #Python
```

**Examples** :  
ee.Geometry  
ee.Feature  
ee.FeatureCollection  
ee.Image  
ee.ImageCollection

### a. Geometries

In [ ]:
# -------------------------  Declaration -----------------------------
point = ee.Geometry.Point(2.3488,48.8534) #(longitude,latitude)

multiPoint = ee.Geometry.MultiPoint(0, 45, 5, 6, 70, -56)

lineString = ee.Geometry.LineString([[0, 45], [5, 6], [70, -56]])

multiLineString = ee.Geometry.MultiLineString([[[0, 45], [5, 6], [70, -56]], [[0, -45], [-5, -6], [-70, 56]]])

linearRing = ee.Geometry.LinearRing(0, 45, 5, 6, 70, -56, 0, 45)

rectangle = ee.Geometry.Rectangle(0, 0, 60, 30)  #(xmin,ymin,xmax,ymax)

polygon = ee.Geometry.Polygon([[[0, 0], [6, 3], [5, 5], [-30, 2], [0, 0]]])

multiPolygons = ee.Geometry.MultiPolygon([ee.Geometry.Polygon([[0, 0], [6, 3], [5, 5], [-30, 2], [0, 0]]),ee.Geometry.Polygon([[0, 0], [-6, -3], [-5, -5], [30, -2], [0, 0]])])

# -------------------------  Operations -----------------------------
# Finding the length of a line
linLen = lineString.length()

# Finding the perimeter of a geometry
geoPeri = geometry.perimeter()

# Reducing number of vertices in geometry
simpGeo = geometry.simplify(100)

# Finding the centroid of a geometry
centrGeo = geometry.centroid()

# Creating buffer around a geometry
buffGeo = geometry.buffer(100) #in meters

# Finding the bounding rectangle of a geometry
bounGeo = geometry.bounds()

# Finding the smallest polygon that can envelope a geometry
convexGeo = geometry.convexHull()

# Finding common areas between two or more geometries
interGeo = geometry1.intersection(geometry2)

# Finding the area that includes two or more geometries
unGeo = geometry1.union(geometry2)


### b. Features and FeatureCollections
* Features are geometries associated with specific properties.  
* Feature collections are groups of features.

In [ ]:
# Load US county dataset and filter the counties that are in Connecticut.
countyData=ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '09'))

# Creating a feature with a specific property value
feat = ee.Feature(geometry, {'Name': 'featureName', 'Size': 500})

# Creating a feature from an existing feature, renaming a property
featNew = feature.select(['name'], ['descriptor'])

# Extracting values of a property from a Feature
featVal = feature.get('size')


### c. Images and ImageCollections

In [ ]:
# ---------------------------- Declaration -------------------------------
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2020-01-01','2020-12-31').filterRegion(rectangle).filterMetadata('CLOUD_COVER','less_than',0.3)
image = collection.median() #or image = ee.Image('AssetId') if the image exists in the database

#------------------------ Operations on images ---------------------------

# Selecting the bands of an image
band = image.select(bandName) #example : image.select('B1')

# Creating masks
mask = image.eq(value); #or .neq or .gt or .gte or .lt or .lte

# Applying image masks
masked = image.updateMask(mask)

# Performing pixelwise calculations
results = image.add(value) #or .subtract , .multiply , .divide , .max , .min , .abs , .round , .floor , .ceil , .sqrt , .exp, .log, .log10, .sin , .cos , .tan , .sinh , .cosh , .tanh , .acos, .asin

# Shift pixels of an image
newImage = oldImage.leftShift(valueOfShift) #or .rightShift

# Reducing an image to a statistic for an area of interest
outputDictionary = varImage.reduceRegion(reducer, geometry, scale)  #see the list of reducers in ee. documentation

#-------------------- Operations on image collections ----------------------

# Selecting the first n images in a collection (based on property)
selectedImages = imCollection.limit(n, propertyName, order)

# Selecting images based on particular properties
selectedIm = imCollection.filterMetadata(propertyName, operator, value) #Operators include: “equals”, “less_than”, “greater_than”, “not_equals”, “not_less_than”, “not_greater_than”, “starts_with”, “ends_with”, “not_starts_with”, “not_ends_with”, “contains”, “not_contains”.
##example : collection.filterMetadata('CLOUD_COVER','less_than',0.3)

# Selecting images within a date range
selectedIm = imCollection.filterDate(startDate, stopDate)

# Selecting images within a bounding geometry
selectedIm = imCollection.filterBounds(geometry)

# Performing pixelwise calculations for all images in a collection
sumOfImages = imCollection.sum() #or product(), max(), min(), mean(), mode(), median(), count().

# Alternatively, using reducers:
sumOfImages = imCollection.reduce(ee.Reducer.sum());

# Compositing images in collection with the last image on top
mosaicOfImages = imCollection.mosaic();

# Alternatively, using reducers:
sumOfImages = imCollection.reduce(ee.Reducer.first());


### d. Other types

In [ ]:
# Other types
string = ee.String('This is a string')
number = ee.Number(5);
array = ee.Array([[5, 2, 3], [-2, 7, 10], [6, 6, 9]])
liste = ee.List([5, 'five', 6, 'six'])
dictionary = ee.Dictionary({'five': 5, 'six': 6})

## 2. Filtering

In [ ]:
# Filtering by property values
bFilter = ee.Filter.eq(propertyName, value) #or .neq , .gt , .gte , .lt , and .lte

# Filtering based on maximum difference from a threshold
diffFilter = ee.Filter.maxDifference(threshold, propertyName, value)

# Filtering by text property
var txtFilter = ee.Filter.stringContains(propertyName, stringValue) #or .stringStartsWith, and .stringEndsWith

# Filtering by a value range
var rangeFilter = ee.Filter.rangeContains(propertyName, stringValue, minValue, maxValue)

# Filtering by specific property values
var listFilter = ee.Filter.listContains(propertyName, value1, propertyName2, value2) #.inList to test against a list of values

# Filtering by date range
var dateFilter = ee.Filter.calendarRange(startDate, stopDate)

# Filtering by particular days of the year
dayFilter = ee.Filter.dayOfYear(startDay, stopDay)

# Filtering by a bounding area
boundsFilter = ee.Filter.bounds(geometryOrFeature)

# Combining and inversing filters
newFilterAnd = ee.Filter.and(listOfFilters)
newFilterOr = ee.Filter.or(listOfFilters)
inverseFilter = ee.Filter.not(filter)

## 3. Mapping

In [ ]:
import geemap.foliumap as emap
Map = emap.Map()

Map.setCenterObject(point,10) #or Map.setCenter(long, lat, zoomLevel), Zoom level varies from 0 (no zoom) to 20 (highest zoom level)

# The following params are for landsat images only, check min/max of images if other source of image (Exemple min:0,max:0.3 for sentinel2)
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 1,
  'gamma': 1.4,
}

Map.addLayer(image,visParams,'landsat image')

Map

## 4. Displaying Metadata

In [ ]:
print(image.getInfo()) 
#Be careful, if the image is too big (no boundaries for example) you will have an error due to memory (you can't print more than 5,000 elements at once) try print(image.clip(region).getInfo()) instead
print(rectangle.getInfo())

## 5. Exporting Data

In [ ]:
Export.image.toDrive({
  'collection': varImage, 
  'description': 'fileName', 
  'region': geometry, 
  'scale': 1000
})

#or Export.image.toCloudStorage(), Export.image.toAsset(), Export.table.toDrive(), Export.table.toCloudStorage(), Export.video.toCloudStorage(), Export.video.toDrive().

## 6. Example : Exporting a Sentinel1 tif Image 
The image is taken from a window of a given center and radius

In [ ]:
def get_sentinel1(date_min,date_max,longitude,latitude,radius):
  # Image collection filtering 
  collection = ee.ImageCollection("COPERNICUS/S1_GRD")
  collection = collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  collection = collection.select('VV','VH').filterDate(date_min,date_max)

  # Image definition
  image = collection.median()
  # When you take the median, the projection of the image changes, use image.setDefaultProjection to change the projection and image.projection to find the projection
  image = image.setDefaultProjection(collection.first().projection())
  point = ee.Geometry.Point([longitude,latitude])
  image = image.clip(point.buffer(radius))

  # Bands processing
  band1 = image.select('VV')
  band2 = image.select('VH')
  band3 = image.select('VV').subtract(image.select('VH'))
  image = image.cat([band1,band2,band3])
  # Rename bands
  image = image.select(['VV','VH','VV_1'],['VV','VH','VV_VH'])
  return image,point

def download_tif(date_min,date_max,longitude,latitude,name,folder) :
  image,point = get_sentinel1(date_min,date_max,longitude,latitude)
  task= ee.batch.Export.image.toDrive(
      image       = image,
      description = name,
      folder      = folder,
      fileNamePrefix = name,
      region      = image.clip(point).bounds(),
      scale       = 30,
      crs         = 'EPSG:4326'
  )
  task.start()

In [ ]:
download_tif('2020-01-01','2020-12-31',2.3488,48.8534,'sentinel1','data')

# II. Satellite Images Export 
## Table of Contents
* [1. Image Extraction](#1.-Image-Extraction)
* [2. Region Definition](#2.-Region-Definition)
* [3. Export to tif and kml](#3.-Export-to-tif-and-kml)
* [4. Putting Everything Together](#4.-Putting-Everything-Together)

## 1. Image Extraction

In [ ]:
def get_sentinel1(date_min,date_max):
  # Image collection filtering 
  collection = ee.ImageCollection("COPERNICUS/S1_GRD")
  collection = collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  collection = collection.select('VV','VH').filterDate(date_min,date_max)
  # Image definition 
  # You can take the median of the image collection by setting image=collection.median(), or one image by setting image=collection.first()
  image = collection.median() #If you want an image of a particular date, set date_min=date_max (if you are lucky and there's an image taken that day) and set image=collection.first()
  
  # When we take the median, the projection of the image changes, let's recover it 
  image = image.setDefaultProjection(collection.first().projection())

  # Bands processing
  band1 = image.select('VV')
  band2 = image.select('VH')
  band3 = image.select('VV').subtract(image.select('VH')) #or any operation you want
  image = image.cat([band1,band2,band3])
  # Rename the bands
  image = image.select(['VV','VH','VV_1'],['VV','VH','VV_VH'])
  return image

def maskS2clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = ee.Number(2).pow(10).int();
    cirrusBitMask = ee.Number(2).pow(11).int();
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

def get_sentinel2(date_min,date_max):
  # Image collection filtering 
  collection = ee.ImageCollection("COPERNICUS/S2").filterDate(date_min,date_max)

  # Removing clouds
  collection = collection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds)

  # Image definition 
  # You can take the median of the image collection by setting image=collection.median(), or one image by setting image=collection.first()
  image = collection.median()
  
  # When we take the median, the projection of the image changes, let's recover it 
  image = image.setDefaultProjection(collection.first().select('B3').projection())
  return image

def get_ndvi(date_min,date_max):
  # Image collection filtering 
  collection = ee.ImageCollection("COPERNICUS/S2").select('B4','B3','B8','B11').filterDate(date_min,date_max)
  collection = collection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds)

  # Image definition 
  # You can take the median of the image collection by setting image=collection.median(), or one image by setting image=collection.first()
  image = collection.median()
  # When we take the median, the projection of the image changes, let's recover it 
  image = image.setDefaultProjection(collection.first().select('B3').projection())

  # Bands processing
  ndvi = image.normalizedDifference(['B8','B4'])
  ndwi = image.normalizedDifference(['B3','B8'])
  image = image.addBands([ndvi,ndwi])
  # Rename the bands
  image = image.select(['nd','nd_1'],['ndvi','ndwi']) 
  return image

## 2. Region Definition

In [ ]:
def get_region(method,point=None,radius=None,rectangle=None,ftscolname=None):
  '''Returns the region of extraction
  Arguments :
    Method can be either 'point', 'shape' or 'rectangle' :
      - 'point' : returns a square image with center 'point' and radius 'radius' (in meters)
      - 'feature' : returns the entire image that contains the feature collection 'ftscolname'
      - 'rectangle' : returns the image delimited by a rectangle 'rectangle' ie by 4 coordinates [xmin,ymin,xmax,ymax]
    point : a tuple (x,y), required if method = 'point'
    radius : the radius in meters of the square image which center is 'point', required if method = 'point'
    rectangle : an array [xmin,ymin,xmax,ymax] required if method = 'rectangle'
    ftscolname : a string, the name of the shape feature collection.
  Returns : Region, an ee.Geometry
  '''
  if method == 'point' :
    if not radius or not point :
      raise ValueError("Argument 'point' and 'radius' are required if method is 'point'")
    point = ee.Geometry.Point(point)
    region = point.buffer(radius).bounds()
    
  elif method == 'feature' :
    if not ftscolname :
      raise ValueError("Argument 'ftscolname' is required if method is 'shape'")
    shape = ee.FeatureCollection('users/'+ftscolname)
    region = shape.geometry().bounds()

  elif method == 'rectangle' :
    if not rectangle :
      raise ValueError("Argument 'rectangle' is required if method is 'rectangle'")
    rectangle = ee.Geometry.Rectangle(rectangle)
    region = rectangle.geometry().bounds()
  else :
    raise NotImplementedError('This method is not implemented')
  return region

## 3. Export to tif and kml
### Export a tif image 

In [ ]:
def download_tif(image,region,name,folder,dimensions=None,scale=None) :
  '''Exports a tif image.
  Arguments :
    image : ee.Image to export
    region: ee.Geometry, the exported region
    name : the name of the file
    folder : the folder where the tif image will be saved. Could only be a folder (example : 'data'), not a directory. If you want to save the 
      image in a folder in a different directory, execute os.chdir(new_directory) at the beginning of this notebook
    dimensions : an array [height,width], the dimension in pixels of the image. Default to the dimensions of the image, determined by the region and the nominal (or provided) scale.
    scale : float, in meters. Default to the scale of the image, determined by the region and the nominal (or provided) dimensions.
  !Error! if both dimensions and scale are provided
  '''
  if dimensions and scale :
    raise ValueError("You can't provide both dimensions and scale")

  task= ee.batch.Export.image.toDrive(
      image       = image,
      region      = region,
      description = name,
      folder      = folder,
      fileNamePrefix = name,
      crs         = 'EPSG:4326',
      dimensions  = dimensions, #Be careful, if scale=10m, the maximum allowed dimension is [7680,7680]
      scale       = scale
  )
  task.start()

## 4. Putting Everything Together

In [ ]:
def get_tif(source,method,date_min,date_max,folder,point=None,radius=None,rectangle=None,ftscolname=None,dimensions=None,scale=None) :
  import os 
  import time

  # download tif
  print('Processing', source)
  if source == 'sentinel2' :
    fn = get_sentinel2
  elif source == 'sentinel1' :
    fn = get_sentinel1
  elif source == 'ndvi' :
    fn = get_ndvi
  else :
    raise "Image source not available"

  # Get the image and the region
  image  = fn(date_min,date_max)
  region = get_region(method,point,radius,rectangle,ftscolname)

  # Define the name of the tif image
  #date = image.date().format().getInfo()[:10] #only if you used image=collection.first()
  date = date_min.split('-',1)[-1] #only if used image=collection.median() or image=collection.mosaic()
  name = source+'_'+date

  # Download the tif image
  if not os.path.isfile(folder+'/'+name+'.tif') :
    download_tif(image,region,name,folder,dimensions,scale)

In [ ]:
source   = 'sentinel1' #@param ["sentinel1","sentinel2","ndvi"]
method   = 'point' #@param ["point","feature","rectangle"]
date_min = '2020-01-01' #@param {type:'string'}
date_max = '2020-12-31' #@param {type:'string'}
folder   = '' #@param {type:'string'}

point       = None #@param
radius      = None #@param
rectangle   = None #@param
ftscolname  = None #@param
dimensions  = None #@param
scale       = None #@param

get_tif(source,method,date_min,date_max,folder,
        point=point,radius=radius,rectangle=rectangle,ftscolname=ftscolname,
        dimensions=dimensions,scale=scale)

# III. Earth Engine Code Editor Tutorials
The Earth Engine (EE) Code Editor at https://code.earthengine.google.com is a web-based IDE for the Earth Engine JavaScript API. Code Editor features are designed to make developing complex geospatial workflows fast and easy. The Code Editor has the following elements :
* JavaScript code editor
* Map display for visualizing geospatial datasets
* API reference documentation (Docs tab)
* Git-based Script Manager (Scripts tab)
* Console output (Console tab)
* Task Manager (Tasks tab) to handle long-running queries
* Interactive map query (Inspector tab)
* Search of the data archive or saved scripts
* Geometry drawing tools
  
<img src="tuto_images/earth-engine-code-editor.PNG" width=500>

Read https://developers.google.com/earth-engine/guides/playground for more information

## Table of contents : 
* [1. Add Table to Assets](#1.-Add-Table-to-Assets)
* [2. Add Image to Assets](#2.-Add-Image-to-Assets)
* [3. Adding Missclassified Pixels to Assets and Array
](#3.-Adding-Missclassified-Pixels-to-Assets-and-Array)

## 1. Add Table to Assets

- **1.** If your file is a "kml" file, convert it to a ".shp" file by using this website https://mygeodata.cloud/converter/kml-to-shp OR by using **QGIS software** : 
  * **a.** Drag your kml to the QGIS window.
  * **b.** Right-click on your layer and choose "Export" then "Export Feature As"   
  <img src="tuto_images/3.PNG" width="400">  

  * **c.** Set "Format" as "ESRI Shapefile" and the CRS as "EPSG:3857 / Pseudo-Mercator"  
  * **d.** Fill "File name" to the name of the file. Careful, you should provide the directory : example C:\....pipe.shp
  * **e.** Click on "OK" and wait, this could take a moment. Now you have created several files, please keep them all.  
  <img src="tuto_images/4.PNG" width="400"> 
    
- **2.** Upload your files to your Google Earth Engine Editor :   
  * **a.** Go to https://code.earthengine.google.com/
  * **b.** Click on "Assets", then "NEW", then below "Table Upload", click on "Shape files". 
  <img src="tuto_images/5.PNG" width="400">
  * **c.** Select all the files you just created with QGIS.  
  * **d.** Name your Table, for example "saur_zone2"
  * **e.** Click on "UPLOAD" 
  <img src="tuto_images/16.PNG" width="400">
  - **f.** On the right corner of your screen, click on "Tasks". Check the status of your export.
  <img src="tuto_images/17.PNG" width="400">
 
Now you can access your table by typing : image = ee.FeatureCollection(assetid) with assetid = directory/network_name.
You can find the assetid by clicking on the asset.  
<img src="tuto_images/11.PNG?modified=02022021134100" width="400">

## 2. Add Image to Assets
*replace `filename` by the name of your inference image in the tutorial below*

- **1.** First, if not already done, add your network to Google Earth Editor's Assets following the tutorial in the previous section.
- **2.** Go to https://code.earthengine.google.com/ .
- **3.** Click on "Assets", then "NEW", then below "Image Upload", click on "GeoTIFF".  
<img src="tuto_images/1.PNG" width="400">
- **4.** In "Sources files" select the files `pred_filename.TFRecord` and `filename-mixer.json` in your folder 'inference'
- **5.** Set "AssetId" to "`filename`_pred"
- **6.** Click on "UPLOAD"  z
<img src="tuto_images/2.PNG" width="400">
- **7.** On the right corner of your screen, click on "Tasks". Check the status of your export.  
If there's an error "cannot read mixer file", retry the steps above by putting the mixer file before the tfrecord file and vise-versa several times until you succeed, the system bugs sometimes.
<img src="tuto_images/6.PNG" width="400">

Now you can access your image by typing : image = **ee.Image(assetid)** with assetid = directory/filename_pred.  
You can find the assetid by clicking on the asset.  
<img src="tuto_images/7.PNG?modified=02022021134700" width="400">

## 3. Adding Missclassified Pixels to Assets and Array (after first training)
* **1.** Go the https://code.earthengine.google.com/ and display the predictions on the Map by executing the following in the Code Editor :
``` 
var imageId ='users/leakm/filename_pred'
image = ee.Image(imageId)
Map.addLayer(image,{min:0,max:3,palette:['lime','darkgreen','yellow','blue']},'predictions') 
``` 
* **2.** Clic on the 'Satellite' button on the right corner to visualize satellite images. 
<img src="tuto_images/12.PNG?modified=02022021135200" width="500">  
* **3.** Zoom in the image, play with the transparency slider (where the "Layers" button is) and spot the misclassified regions.
* **4.** Clic on the Rectangle button on the left and draw rectangles around all the misclassified regions **of one and only one class**. Example : draw **only the fields** that were misclassified.
<img src="tuto_images/13.PNG" width="500">  
* **5.** Clic on the edit button of the given geometry. 
<img src="tuto_images/14.PNG" width="500">  
* **6.** Change the name of the feature  
* **7.** Change the type from Geometry to FeatureCollection.
* **8.** Add a property called "landcover" and set its value to the class' index (0 for field, 1 for forest, 2 for urban and 3 for water)
* **9.** Save  
<img src="tuto_images/15.PNG" width="500">  
* **10.** Do the same for the other misclassified classes.  
* **11.** Let's say we constructed 'misclassified_fields' and 'misclassified_urbain'. Now we have to join the FeatureCollections and export the result. Write this code on the Editor and execute :  
```
var new_pixels = miclassified_fields.merge(misclassified_urbain)` 
Export.table.toAsset({  
  collection:new_pixels,  
  description:'misclassified_pixels',  
  assetId:'misclassified_pixels'  
})
```
* **12.** Now back to your Python script. Execute the following function to get a tuple (new_pixels,new_labels).



In [ ]:
def get_more_pixels(collection_name,start_date,end_date):
  geometry = ee.FeatureCollection(collection_name)
  list = geometry.toList(geometry.size())
  image = get_image(start_date,end_date) #the same image source as the training set
  pixels = []
  labels = []
  for i in range(list.size().getInfo()) :
    new_class = ee.Feature(list.get(i))
    array = image.sampleRectangle(new_class.geometry(),defaultValue=0)
    im = np.dstack([np.array(array.get(b).getInfo()) for b in BANDS])
    im = np.reshape(im,[-1,im.shape[-1]])
    pixels.append(im)
    labels.append(np.int64(np.ones(im.shape[0])*new_class.get('landcover').getInfo()))

  pixels = np.concatenate(pixels,axis=0)
  labels = np.concatenate(labels,axis=0)
  pixels = feature_process(pixels) #the same processing as for the training set
  return pixels,labels

new_pixels,new_labels = get_more_pixels('users/Leakmited/misclassified_pixels','2020-01-01','2020-12-31')
#features = np.concatenate([pixels,new_pixels],axis=0)
#labels = np.concatenate([labels,new_labels],axis=0)